# Train Pixel Level Annotation Model

### This notebook uses pixel level annotations to train a Random Forest Classifier to predict labels

We assume Pixel level annotations are available, as produced by the "../data/annotations/transform_polygon_annotations_to_pixels.ipynb" notebook. Feature selection and grid_search for optimal parameters has been done in a separate notebook ("Coepelduynen/make_train_model_on_annotations_coepelduynen.ipynb") and those outcomes are taken as given in this notebook.

Change the set Variables cell below as desired and then run the entire notebook to get cross_validation results as well as a final model trained on all data.

Date: 2024-01-12\
Author: Pieter Kouyzer, Michael de Winter

In [1]:
%load_ext autoreload
%autoreload 2

In [33]:
%matplotlib notebook
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import pprint

from training.train import train_imbalanced_model, cross_validation_balance_on_date, create_folds
from training.utils import get_cross_validation_results_filepath, get_model_filepath
from training.metric_calculation import calculate_average_metrics, get_metrics

In [34]:
# Set Variables
location = "Schippersgat"
satellite_constellation = "Superview"

if location == "Voornes Duin":
    annotated_pixels_filepath = "C:/Users/pzhadmin/Data/remote-sensing/annotations/Voornes_Duin_PNEO_2024-01-29_pixel_annotations.parquet"
elif location == "Coepelduynen":
    annotated_pixels_filepath = "C:/repos/satellite-images-nso-datascience/data/annotations/annotations_pixel_dataframes/annotaties_coepelduynen_to_pixel_2023.parquet"

elif location == "Schippersgat":
    if satellite_constellation == "PNEO":
        annotated_pixels_filepath = "C:/repos/satellite-images-nso-datascience/data/annotations/annotations_pixel_dataframes/PNEO_waterplanten_annotations.parquet"
    if satellite_constellation == "Superview":
        annotated_pixels_filepath = "C:/repos/satellite-images-nso-datascience/data/annotations/annotations_pixel_dataframes/Superview_waterplanten_annotations.parquet"

In [35]:
# Optimal parameters and features Voornes
if location == "Voornes Duin":
    selected_features = ['r', 'g', 'b', 'n', 'e', 'd', 'ndvi','re_ndvi', 'height']
    optimal_parameters = {
        "n_estimators": 10, 
        "min_samples_split": 5, 
        "min_samples_leaf": 1,
        "max_features": "auto", 
        "bootstrap": False
    }

elif location == "Coepelduynen":
    # Optimal parameters and features
    selected_features = ['r','g','b','n','e','d','ndvi','re_ndvi']
    optimal_parameters = {
        "n_estimators": 10, 
        "min_samples_split": 5, 
        "min_samples_leaf": 1,
        "bootstrap": False
    }

elif location == "Schippersgat":
    # Optimal parameters and features
    selected_features = ['r', 'g', 'b', 'i', 'ndvi', 'ndwi']
    optimal_parameters = {
        "n_estimators": 10, 
        "min_samples_split": 5, 
        "min_samples_leaf": 1,
        "bootstrap": False
    }

In [36]:
df = pd.read_parquet(annotated_pixels_filepath)
df

,r,g,b,i,ndvi,label,image,date,season,annotation_no,ndwi
0,1171.0,753.0,630.0,433.0,126.0,Water,20220705_110002_SV2-01_SV_RD_11bit_RGBI_50cm_W...,20220705_110002,Summer,0_20220705_annotations,19.0
1,1107.0,693.0,562.0,386.0,128.0,Water,20220705_110002_SV2-01_SV_RD_11bit_RGBI_50cm_W...,20220705_110002,Summer,0_20220705_annotations,20.0
2,1101.0,687.0,558.0,383.0,128.0,Water,20220705_110002_SV2-01_SV_RD_11bit_RGBI_50cm_W...,20220705_110002,Summer,0_20220705_annotations,20.0
3,1180.0,762.0,640.0,440.0,126.0,Water,20220705_110002_SV2-01_SV_RD_11bit_RGBI_50cm_W...,20220705_110002,Summer,0_20220705_annotations,19.0
4,1174.0,757.0,636.0,437.0,126.0,Water,20220705_110002_SV2-01_SV_RD_11bit_RGBI_50cm_W...,20220705_110002,Summer,0_20220705_annotations,19.0
...,...,...,...,...,...,...,...,...,...,...,...
92895,349.0,265.0,164.0,164.0,123.0,Water,20190302_113613_SV1-02_50cm_RD_11bit_RGBI_Mijd...,20190302_113613,Spring,4_20190302_annotations,52.0
92896,349.0,265.0,164.0,164.0,123.0,Water,20190302_113613_SV1-02_50cm_RD_11bit_RGBI_Mijd...,20190302_113613,Spring,4_20190302_annotations,52.0
92897,348.0,264.0,163.0,163.0,123.0,Water,20190302_113613_SV1-02_50cm_RD_11bit_RGBI_Mijd...,20190302_113613,Spring,4_20190302_annotations,52.0
92898,351.0,266.0,165.0,165.0,123.0,Water,20190302_113613_SV1-02_50cm_RD_11bit_RGBI_Mijd...,20190302_113613,Spring,4_20190302_annotations,51.0


In [37]:
# This is to give an indication of the amount of data points per image
df['date'].value_counts()

date
20220705_110002    111662
20210617_111532     95779
20190302_113613     92900
Name: count, dtype: int64

In [38]:
# This is to give an indication of the amount of data points per label
df['label'].value_counts()

label
Water          207319
Waterplants     52108
Ground          40914
Name: count, dtype: int64

In [14]:
# Make sure there is no Ashpalt.
df = df[df['label'] != "Asphalt"]

### Cross Validation

We do cross-validation, where the folds are decided by the 'date' column. This is to avoid pixels from the same image from ending up in both the train and test datasets. We display the metrics averaged over the folds and write the results to a pickle.

In [39]:
model = RandomForestClassifier(**optimal_parameters)
scaler = StandardScaler()

In [40]:
if location == "Voornes Duin":
    cv =5 
    random_state = 1337
    sampling_type_boundary = 100000
elif location == "Coepelduynen":
    cv =4 
    random_state = 1337
    sampling_type_boundary = 898609
elif location == "Schippersgat":
    cv =3
    random_state = 1337
    sampling_type_boundary = 898609

In [41]:
df['date'] = df['date'].str.split('_').str[0]

In [42]:
results = cross_validation_balance_on_date(data=df, model=model, cv=cv, features=selected_features, random_state=random_state, sampling_type_boundary=sampling_type_boundary , scaler=scaler)

---------fold: 1
Picked hold out dates: 
['20220705']
Oversampling to rebalance dataset


2024/05/21 14:19:20 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '0bb572ae125f4c09a7f1569289d71268', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/05/21 14:19:20 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2024/05/21 14:19:20 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not have a `predict` or `transform` function, which is required in order to infer the signature
2024/05/21 14:19:20 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024/05/21 14:19:21 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\pzhadmin\AppData\Local\Temp\2\tmpip9spbix\model\mode

Fitting model


<IPython.core.display.Javascript object>

2024/05/21 14:19:44 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\pzhadmin\AppData\Local\Temp\2\tmp9uea_en0\model\model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.4.0', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback.


...

Fitting model


<IPython.core.display.Javascript object>

2024/05/21 14:20:38 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\pzhadmin\AppData\Local\Temp\2\tmpauvj5dx4\model\model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.4.0', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback.


Calculating train metrics
Calculating test metrics
{'Ground': {'precision': nan, 'recall': 0.0, 'f1-score': nan, 'support': 12568}, 'Water': {'precision': 0.000951977998730696, 'recall': 0.00011203505452372654, 'f1-score': 0.00020047668901610496, 'support': 80332}, 'Waterplants': {'precision': 0.0, 'recall': nan, 'f1-score': nan, 'support': 0}}


c:\ProgramData\Anaconda3\envs\py310\envs\py312\Lib\site-packages\training\metric_calculation.py:28: RuntimeWarning: invalid value encountered in scalar divide
  precision = confusion_matrix[i, i] / confusion_matrix[:, i].sum()
c:\ProgramData\Anaconda3\envs\py310\envs\py312\Lib\site-packages\training\metric_calculation.py:29: RuntimeWarning: invalid value encountered in scalar divide
  recall = confusion_matrix[i, i] / confusion_matrix[i, :].sum()


In [43]:
calculate_average_metrics(results=results)

,precision,recall,f1-score
Ground,0.367761,0.011292,0.021911
Water,0.485410,0.493713,0.458389
Waterplants,0.834312,0.451601,0.351990


In [44]:
cross_validation_results_filepath = get_cross_validation_results_filepath(location=location, satellite_constellation=satellite_constellation, df=df)
print(f"Saving to {cross_validation_results_filepath}")
with open(cross_validation_results_filepath, "wb") as file:
    pickle.dump(results, file)

Saving to ../saved_models/Superview_Schippersgat_20190302_to_20220705_cross_validation_results.pkl


### Export Definitive model.

Trains a Random Forest Classifier model on all data and writes it to a pickle file for later use. This is the definitive model output by this notebook.

In [45]:
final_model = RandomForestClassifier(**optimal_parameters)
final_scaler = StandardScaler()

train_imbalanced_model(
    X_train=df[selected_features], 
    y_train=df["label"], 
    model=final_model, 
    random_state=1337, 
    sampling_type_boundary=sampling_type_boundary ,
    scaler=final_scaler
)
pprint.pprint(get_metrics(y=df["label"], X=df[selected_features], model=final_model, scaler=final_scaler))

Oversampling to rebalance dataset


2024/05/21 14:27:34 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'affdd345295344c0983f39c37dd870d3', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/05/21 14:27:34 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2024/05/21 14:27:34 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not have a `predict` or `transform` function, which is required in order to infer the signature
2024/05/21 14:27:34 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024/05/21 14:27:35 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\pzhadmin\AppData\Local\Temp\2\tmpgwlwwda1\model\mode

Fitting model


<IPython.core.display.Javascript object>

2024/05/21 14:28:11 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\pzhadmin\AppData\Local\Temp\2\tmp22tbj1ax\model\model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.4.0', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback.


{'Ground': {'f1-score': 1.0, 'precision': 1.0, 'recall': 1.0, 'support': 40914},
 'Water': {'f1-score': 1.0, 'precision': 1.0, 'recall': 1.0, 'support': 207319},
 'Waterplants': {'f1-score': 1.0,
                 'precision': 1.0,
                 'recall': 1.0,
                 'support': 52108}}


In [46]:
final_artefact = {
    "model": final_model,
    "scaler": final_scaler
}

In [47]:
final_model_filepath = get_model_filepath(location=location, satellite_constellation=satellite_constellation, df=df)
print(f"Saving to {final_model_filepath}")
with open(final_model_filepath, "wb") as file:
    pickle.dump(final_artefact, file)

Saving to ../saved_models/Superview_Schippersgat_20190302_to_20220705_random_forest_classifier.sav
